# https://pubs.acs.org/doi/pdf/10.1021/acs.jmedchem.7b00696

In [1]:
from glob import glob
from e3fp.conformer.util import smiles_to_dict
from e3fp.config.params import default_params
from e3fp.pipeline import fprints_from_smiles
from e3fp.pipeline import confs_from_smiles
from e3fp.pipeline import params_to_dicts
from e3fp.config.params import read_params
from e3fp.config.params import default_params
from python_utilities.parallel import Parallelizer
import pickle
import os
import time

In [2]:
import pandas as pd
import numpy as np

In [3]:
from itertools import islice

def chunks(data, SIZE=10000):
    it = iter(data)
    for i in range(0, len(data), SIZE):
        yield {k:data[k] for k in islice(it, SIZE)}

In [4]:
%%capture
from tqdm.notebook import tqdm
tqdm().pandas()

In [5]:
os.chdir('/tf/notebooks/code_for_pub/e3fp_fps')

In [8]:
with open('/tf/notebooks/code_for_pub/smiles_files/smiles_drugcomb_BY_cid_duplicated.pickle', 'rb') as f: # protonated?
    sms = pickle.load(f)

In [7]:
sms

1                                 O=c1[nH]cc(F)c(=O)[nH]1
2                      CC1(c2nc3c(C(N)=O)cccc3[nH]2)CCCN1
3       C=CCn1c(=O)c2cnc(Nc3ccc(N4CCN(C)CC4)cc3)nc2n1-...
4       Cn1c(=O)n(-c2ccc(C(C)(C)C#N)cc2)c2c3cc(-c4cnc5...
5          CC(C)CC(NC(=O)C(Cc1ccccc1)NC(=O)c1cnccn1)B(O)O
                              ...                        
4446    COc1ccc(S(=O)(=O)N(C(C)=O)c2ccccc2C=Cc2cc[n+](...
4496    CNC(=O)c1c(F)cccc1Nc1nc(Nc2cc3c(cc2OC)CCN3C(=O...
4501    CCN1CCN(Cc2ccc(NC(=O)c3ccc(C)c(C=Cc4cnc5[nH]cc...
4526    COC(=O)C=CC(=O)N(O)CCCCNCc1ccc(COC(=O)Nc2cccc3...
4543    CC1(CO)CN(c2cc(C(=O)Nc3ccc4c(c3)-c3c(c(C(N)=O)...
Name: smiles, Length: 4153, dtype: object

In [11]:
confgen_params, fprint_params = params_to_dicts(default_params)


In [13]:
fprint_params

{'bits': 1024,
 'level': 5,
 'first': 3,
 'radius_multiplier': 1.718,
 'stereo': True,
 'counts': False,
 'include_disconnected': True,
 'rdkit_invariants': False,
 'remove_duplicate_substructs': True,
 'exclude_floating': True}

In [12]:
confgen_params

{'num_conf': -1,
 'first': -1,
 'pool_multiplier': 1,
 'rmsd_cutoff': 0.5,
 'max_energy_diff': None,
 'forcefield': 'uff',
 'out_dir': 'conformers',
 'compress': 2,
 'seed': -1,
 'standardise': False,
 'protonate': False}

In [14]:
del confgen_params['protonate']
del confgen_params['standardise']

In [7]:
fprint_params['include_disconnected']=True
fprint_params['stereo']=False
fprint_params['first']=1
confgen_params['first']=20

kwargs = {"confgen_params": confgen_params, 
          "fprint_params": fprint_params
         }


In [11]:
# generate E3FP no_Stereo

smiles_dict =sms.to_dict()
it = chunks(smiles_dict, 100) # need to chunk, because hanging up

l = []
counter = 0

for i in tqdm(it,total=42):
    parallelizer = Parallelizer(parallel_mode='processes', num_proc = 10) # 1149.8sec with processes vs 1440.6 with threads on 100 smiles. 
    smiles_iter = ((smiles, name) for name, smiles in i.items())
    fprints_list = parallelizer.run(fprints_from_smiles, smiles_iter, kwargs=kwargs) 
    with open(f'./save/chunk_{counter}', 'wb') as f:
        pickle.dump(fprints_list, f)
    counter += 1
    l.append(fprints_list)

2020-10-22 16:30:55,678|INFO|Parallelizer initialized with mode 'processes' and 10 processors.
2020-10-22 16:30:55,798|INFO|Generating conformers for 3.
2020-10-22 16:30:55,794|INFO|Generating conformers for 2.
2020-10-22 16:30:55,810|INFO|Generating conformers for 1.
2020-10-22 16:30:55,820|INFO|Generating conformers for 5.
2020-10-22 16:30:55,828|INFO|Generating conformers for 4.
2020-10-22 16:30:55,825|INFO|Generating conformers for 10.
2020-10-22 16:30:55,813|INFO|Generating conformers for 6.
2020-10-22 16:30:55,818|INFO|Generating conformers for 8.
2020-10-22 16:30:55,834|INFO|Generating conformers for 7.
2020-10-22 16:30:55,825|INFO|Generating conformers for 9.
2020-10-22 16:30:56,015|INFO|Generated 1 conformers for 1.
2020-10-22 16:30:56,017|INFO|Generating fingerprints for 1.
2020-10-22 16:30:56,033|INFO|Generated 1 fingerprints for 1.
2020-10-22 16:30:56,039|INFO|Generating conformers for 11.
2020-10-22 16:30:56,858|INFO|Generated 1 conformers for 7.
2020-10-22 16:30:56,860|IN

2020-10-22 16:32:00,334|INFO|Generating fingerprints for 11.
2020-10-22 16:32:00,386|INFO|Generated 1 fingerprints for 11.
2020-10-22 16:32:00,389|INFO|Generating conformers for 42.
2020-10-22 16:32:00,696|INFO|Generated 1 conformers for 42.
2020-10-22 16:32:00,699|INFO|Generating fingerprints for 42.
2020-10-22 16:32:00,718|INFO|Generated 1 fingerprints for 42.
2020-10-22 16:32:00,722|INFO|Generating conformers for 43.
2020-10-22 16:32:01,864|INFO|Generated 1 conformers for 41.
2020-10-22 16:32:01,866|INFO|Generating fingerprints for 41.
2020-10-22 16:32:01,880|INFO|Generated 1 fingerprints for 41.
2020-10-22 16:32:01,883|INFO|Generating conformers for 44.
2020-10-22 16:32:02,376|INFO|Generated 1 conformers for 40.
2020-10-22 16:32:02,378|INFO|Generating fingerprints for 40.
2020-10-22 16:32:02,393|INFO|Generated 1 fingerprints for 40.
2020-10-22 16:32:02,399|INFO|Generating conformers for 45.
2020-10-22 16:32:03,556|INFO|Generated 1 conformers for 45.
2020-10-22 16:32:03,562|INFO|Gen

2020-10-22 16:35:06,209|INFO|Generating fingerprints for 77.
2020-10-22 16:35:06,227|INFO|Generated 1 fingerprints for 77.
2020-10-22 16:35:06,231|INFO|Generating conformers for 78.
2020-10-22 16:35:14,809|INFO|Generated 1 conformers for 65.
2020-10-22 16:35:14,813|INFO|Generating fingerprints for 65.
2020-10-22 16:35:14,909|INFO|Generated 1 fingerprints for 65.
2020-10-22 16:35:14,912|INFO|Generating conformers for 79.
2020-10-22 16:35:16,502|INFO|Generated 1 conformers for 79.
2020-10-22 16:35:16,509|INFO|Generating fingerprints for 79.
2020-10-22 16:35:16,533|INFO|Generated 1 fingerprints for 79.
2020-10-22 16:35:16,539|INFO|Generating conformers for 80.
2020-10-22 16:35:18,470|INFO|Generated 1 conformers for 80.
2020-10-22 16:35:18,475|INFO|Generating fingerprints for 80.
2020-10-22 16:35:18,500|INFO|Generated 1 fingerprints for 80.
2020-10-22 16:35:18,506|INFO|Generating conformers for 81.
2020-10-22 16:35:20,066|INFO|Generated 1 conformers for 81.
2020-10-22 16:35:20,073|INFO|Gen

2020-10-22 16:48:41,854|INFO|Generating conformers for 105.
2020-10-22 16:48:41,822|INFO|Generating conformers for 110.
2020-10-22 16:48:41,829|INFO|Generating conformers for 112.
2020-10-22 16:48:41,847|INFO|Generating conformers for 111.
2020-10-22 16:48:41,835|INFO|Generating conformers for 113.
2020-10-22 16:48:42,201|INFO|Generated 1 conformers for 108.
2020-10-22 16:48:42,203|INFO|Generating fingerprints for 108.
2020-10-22 16:48:42,222|INFO|Generated 1 fingerprints for 108.
2020-10-22 16:48:42,228|INFO|Generating conformers for 115.
2020-10-22 16:48:42,438|INFO|Generated 1 conformers for 109.
2020-10-22 16:48:42,444|INFO|Generating fingerprints for 109.
2020-10-22 16:48:42,472|INFO|Generated 1 fingerprints for 109.
2020-10-22 16:48:42,479|INFO|Generating conformers for 116.
2020-10-22 16:48:42,782|INFO|Generated 1 conformers for 113.
2020-10-22 16:48:42,784|INFO|Generating fingerprints for 113.
2020-10-22 16:48:42,808|INFO|Generated 1 fingerprints for 113.
2020-10-22 16:48:42,81

2020-10-22 16:49:27,064|INFO|Generating conformers for 140.
2020-10-22 16:49:28,594|INFO|Generated 1 conformers for 137.
2020-10-22 16:49:28,596|INFO|Generating fingerprints for 137.
2020-10-22 16:49:28,622|INFO|Generated 1 fingerprints for 137.
2020-10-22 16:49:28,628|INFO|Generating conformers for 141.
2020-10-22 16:49:29,031|INFO|Generated 1 conformers for 140.
2020-10-22 16:49:29,033|INFO|Generating fingerprints for 140.
2020-10-22 16:49:29,058|INFO|Generated 1 fingerprints for 140.
2020-10-22 16:49:29,061|INFO|Generating conformers for 142.
2020-10-22 16:49:31,671|INFO|Generated 1 conformers for 141.
2020-10-22 16:49:31,673|INFO|Generating fingerprints for 141.
2020-10-22 16:49:31,700|INFO|Generated 1 fingerprints for 141.
2020-10-22 16:49:31,702|INFO|Generating conformers for 143.
2020-10-22 16:49:43,310|INFO|Generated 1 conformers for 142.
2020-10-22 16:49:43,314|INFO|Generating fingerprints for 142.
2020-10-22 16:49:43,346|INFO|Generated 1 fingerprints for 142.
2020-10-22 16:49

...

2020-10-23 16:57:03,341|INFO|Generating fingerprints for 3986.
2020-10-23 16:57:03,368|INFO|Generated 1 fingerprints for 3986.
2020-10-23 16:57:10,261|INFO|Generated 1 conformers for 3940.
2020-10-23 16:57:10,265|INFO|Generating fingerprints for 3940.
2020-10-23 16:57:10,324|INFO|Generated 1 fingerprints for 3940.
2020-10-23 16:57:37,760|INFO|Generated 1 conformers for 3981.
2020-10-23 16:57:37,768|INFO|Generating fingerprints for 3981.
2020-10-23 16:57:37,840|INFO|Generated 1 fingerprints for 3981.
2020-10-23 16:57:42,718|INFO|Generated 1 conformers for 3983.
2020-10-23 16:57:42,721|INFO|Generating fingerprints for 3983.
2020-10-23 16:57:42,785|INFO|Generated 1 fingerprints for 3983.
2020-10-23 16:58:01,345|INFO|Generated 1 conformers for 3959.
2020-10-23 16:58:01,349|INFO|Generating fingerprints for 3959.
2020-10-23 16:58:01,449|INFO|Generated 1 fingerprints for 3959.
2020-10-23 16:58:01,669|INFO|Generated 1 conformers for 3965.
2020-10-23 16:58:01,675|INFO|Generating fingerprints fo

2020-10-23 17:01:27,683|INFO|Generating fingerprints for 4000.
2020-10-23 17:01:27,737|INFO|Generated 1 fingerprints for 4000.
2020-10-23 17:01:27,741|INFO|Generating conformers for 4021.
2020-10-23 17:01:41,614|INFO|Generated 1 conformers for 4008.
2020-10-23 17:01:41,620|INFO|Generating fingerprints for 4008.
2020-10-23 17:01:41,676|INFO|Generated 1 fingerprints for 4008.
2020-10-23 17:01:41,681|INFO|Generating conformers for 4022.
2020-10-23 17:01:51,399|INFO|Generated 1 conformers for 4014.
2020-10-23 17:01:51,403|INFO|Generating fingerprints for 4014.
2020-10-23 17:01:51,451|INFO|Generated 1 fingerprints for 4014.
2020-10-23 17:01:51,455|INFO|Generating conformers for 4023.
2020-10-23 17:01:52,418|INFO|Generated 1 conformers for 4021.
2020-10-23 17:01:52,423|INFO|Generating fingerprints for 4021.
2020-10-23 17:01:52,472|INFO|Generated 1 fingerprints for 4021.
2020-10-23 17:01:52,476|INFO|Generating conformers for 4024.
2020-10-23 17:01:52,888|INFO|Generated 1 conformers for 4024.


2020-10-23 17:03:15,077|INFO|Generating fingerprints for 4049.
2020-10-23 17:03:15,107|INFO|Generated 1 fingerprints for 4049.
2020-10-23 17:03:15,110|INFO|Generating conformers for 4054.
2020-10-23 17:03:15,270|INFO|Generated 1 conformers for 4052.
2020-10-23 17:03:15,274|INFO|Generating fingerprints for 4052.
2020-10-23 17:03:15,304|INFO|Generated 1 fingerprints for 4052.
2020-10-23 17:03:15,309|INFO|Generating conformers for 4055.
2020-10-23 17:03:15,582|INFO|Generated 1 conformers for 4053.
2020-10-23 17:03:15,585|INFO|Generating fingerprints for 4053.
2020-10-23 17:03:15,612|INFO|Generated 1 fingerprints for 4053.
2020-10-23 17:03:15,615|INFO|Generating conformers for 4056.
2020-10-23 17:03:16,909|INFO|Generated 1 conformers for 4055.
2020-10-23 17:03:16,912|INFO|Generating fingerprints for 4055.
2020-10-23 17:03:16,963|INFO|Generated 1 fingerprints for 4055.
2020-10-23 17:03:16,969|INFO|Generating conformers for 4057.
2020-10-23 17:03:20,141|INFO|Generated 1 conformers for 4045.


2020-10-23 17:05:26,775|INFO|Generating fingerprints for 4080.
2020-10-23 17:05:26,817|INFO|Generated 1 fingerprints for 4080.
2020-10-23 17:05:27,822|INFO|Generated 1 conformers for 4069.
2020-10-23 17:05:27,836|INFO|Generating fingerprints for 4069.
2020-10-23 17:05:27,928|INFO|Generated 1 fingerprints for 4069.
2020-10-23 17:05:28,800|INFO|Generated 1 conformers for 4079.
2020-10-23 17:05:28,808|INFO|Generating fingerprints for 4079.
2020-10-23 17:05:28,850|INFO|Generated 1 fingerprints for 4079.
2020-10-23 17:05:29,980|INFO|Generated 1 conformers for 4076.
2020-10-23 17:05:29,991|INFO|Generating fingerprints for 4076.
2020-10-23 17:05:30,039|INFO|Generated 1 fingerprints for 4076.
2020-10-23 17:05:37,747|INFO|Generated 1 conformers for 4086.
2020-10-23 17:05:37,751|INFO|Generating fingerprints for 4086.
2020-10-23 17:05:37,778|INFO|Generated 1 fingerprints for 4086.
2020-10-23 17:05:37,999|INFO|Generated 1 conformers for 4084.
2020-10-23 17:05:38,001|INFO|Generating fingerprints fo

2020-10-23 17:10:43,930|INFO|Generating conformers for 4120.
2020-10-23 17:10:46,762|INFO|Generated 1 conformers for 4120.
2020-10-23 17:10:46,765|INFO|Generating fingerprints for 4120.
2020-10-23 17:10:46,806|INFO|Generated 1 fingerprints for 4120.
2020-10-23 17:10:46,809|INFO|Generating conformers for 4121.
2020-10-23 17:10:46,949|INFO|Generated 1 conformers for 4092.
2020-10-23 17:10:46,956|INFO|Generating fingerprints for 4092.
2020-10-23 17:10:47,015|INFO|Generated 1 fingerprints for 4092.
2020-10-23 17:10:47,019|INFO|Generating conformers for 4122.
2020-10-23 17:10:51,264|INFO|Generated 1 conformers for 4115.
2020-10-23 17:10:51,267|INFO|Generating fingerprints for 4115.
2020-10-23 17:10:51,303|INFO|Generated 1 fingerprints for 4115.
2020-10-23 17:10:51,309|INFO|Generating conformers for 4123.
2020-10-23 17:10:52,439|INFO|Generated 1 conformers for 4121.
2020-10-23 17:10:52,442|INFO|Generating fingerprints for 4121.
2020-10-23 17:10:52,481|INFO|Generated 1 fingerprints for 4121.


In [85]:
l=[]
for i in tqdm(glob("/tf/notebooks/code_for_pub/e3fp_fps/save/*"), total=40):
    
    t = pd.read_pickle(i)
    t_bitness = t[0][0][0].bits # they are all equal. so take first element
    t_df = np.zeros((len(t), t_bitness+1), dtype=np.int16) #+1 for drugcombID
    for ind,ii in enumerate(t):
        dname = ii[1][1]
        if not ii[0]: # if ii[0] False, no fp was generated, fill with -1 all values
            fp = np.full((1024), dtype=np.int16, fill_value=-1)
            fp = np.insert(fp, 0, dname, axis=0)
            t_df[ind]=fp
        else:
            fp = ii[0][0].to_vector(sparse=False,dtype=np.ndarray)
            fp = np.insert(fp, 0, dname, axis=0)
            t_df[ind]=fp
    
    
    l.append(pd.DataFrame(data=t_df[:,1:], index=t_df[:,0]))  
        
df = pd.concat(l)
del l
df = df.sort_index()

In [87]:
df.to_pickle('/tf/notebooks/code_for_pub/fp_files/fps_e3fp_1024bit.pickle')